In [1]:
import numpy as np
import time
import pandas as pd

from game_functions.environment import sbfeatures as sbf
from game_functions.model import brain

Using Theano backend.


In [2]:
env = sbf.Environment()

init


In [3]:
env.controller(initialize=True, close=True)
env.controller(initialize=True, opens=True)
env.action(5, delay=0.1)

5

In [4]:
env.make()

launching game...


In [5]:
env.start_game(botsOn=True, mode='easy')

In [6]:
env.initalize_memory()

hwnd: 22874826
pid: c_long(8272)


In [7]:
print(env.observation())
#env.__grab_addresses__()
#env.__grab_features__()

[100.0, 3.0, -0.8, 100.0, -6.0, 3.2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 3.0, -6.0, -0.8, 3.2]


In [8]:
print(env.p1_score, env.p2_score)

0 0


In [9]:
env.reward_structure(p1_death=-1, p2_death=1, p1_damage=-0.01, p2_damage=0.01)

Reward structure defined!


In [10]:
#print(env.features)
print(env.observation())

[100.0, 3.0, -0.8, 100.0, -6.0, 3.2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0]


In [11]:
print(env.reward(stale_observation=env.features, 
                 new_observation=env.observation()))

0


In [12]:

env.action(5, delay=0.1)

5

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.layers import Dropout


def keras_model(inputs, dropouts=0.2, opti=1e-6, clipping=1, clipval=10):
    keras_model = Sequential()
    keras_model.add(Dense(128, input_dim=inputs, activation='relu'))
    keras_model.add(Dropout(dropouts))
    keras_model.add(Dense(128, activation='relu'))
    keras_model.add(Dropout(dropouts))
    keras_model.add(Dense(1))
    adam = Adam(lr=opti, clipnorm=clipping, clipvalue=clipval)
    keras_model.compile(loss='mse', optimizer='adam')
    return keras_model


In [17]:
input_shape = 17


Qas = keras_model(inputs=input_shape, dropouts=0.2, opti=1e-6, clipping=0.05, clipval=10)

#Qas.load_weights('SB-Qas-v1-20170620.keras')
Qas.load_weights('model_weights//kill_rewards//2800000_model_backup_20170625_2215.keras')

In [22]:
import time
import random
import datetime

episode = 0
training_backup = 500000
save_location = "training_data\\"
training_batch = []

# action settings
randomness = True

#score keeping
p1_score = 0
p2_score = 0

#init#
prev_observation = env.observation()[:]
observation = env.observation()[:]
header = ['episode','p1h', 'p1_x',' p1_y', 'p2h', 'p2_x',' p2_y',
          'right_of', 'left_of', 'above', 'below', 'x_clear', 'y_clear',
          'cd1', 'cd2',
         'aim_down', 'aim_left', 'aim_up', 'aim_right',
         'p1_x_delta', 'p2_x_delta', 'p1_y_delta', 'p2_y_delta',
         'action', 'reward', 'p1_score', 'p2_score']

while True:

        episode += 1
        
        #random action
        if randomness:
            action = random.randint(0, 7)

        #env.action(action, delay=0.1)
        
        observation = env.observation()[:]
        reward = env.reward(stale_observation=prev_observation, 
                            new_observation=observation)
        prev_observation = observation
        
        #datapoint = [episode]
        datapoint = []
        datapoint.extend(observation)
        
        del datapoint[-1]
        del datapoint[-1]
        del datapoint[-1]
        del datapoint[-1]
        del datapoint[3]
        del datapoint[0]
        
        
        predictions = env.create_predictions(datapoint)
        action_rewards = Qas.predict(predictions)
        
        #---action = np.where(advantage == np.amax(advantage))[0][0]
        action = np.where(action_rewards == np.amax(action_rewards))[0][0] # greedy action
        print(action, env.p1_score, env.p2_score)
        
        env.action(action, delay=0.1)
        
        #datapoint.append(action)
        #datapoint.append(reward)
        #datapoint.extend([env.p1_score + 25000, env.p2_score + 25000])
        
        if env.p1_score == 0:
            break
        if env.p2_score == 0:
            break
        
        training_batch.append(datapoint)
        
        if (len(training_batch) % training_backup) == 0:
            name =  save_location + str(episode) + '_training_backup_' + datetime.datetime.today().strftime('%Y%m%d_%H%M') + '.csv'
            exp = pd.DataFrame(training_batch)
            exp.columns = header
            exp.to_csv(name, index=False)
            print('Saving Training Experience...')
            training_batch = []
    
;

4 -20 -20
7 -20 -20
0 -20 -20
1 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
1 -20 -20
0 -20 -20
1 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
7 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
3 -20 -20
1 -20 -20
1 -20 -20
5 -20 -20
5 -20 -20
5 -20 -20
5 -20 -20
5 -20 -20
5 -20 -20
5 -20 -20
5 -20 -20
7 -20 -20
0 -20 -20
0 -20 -20
3 -20 -20
4 -20 -20
6 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
1 -20 -20
6 -20 -20
7 -20 -20
0 -20 -20
2 -20 -20
0 -20 -20
3 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
1 -20 -20
1 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
1 -20 -20
0 -20 -19
0 -20 -20
0 -20 -20
0 -20 -20


0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
1 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
5 -20 -20
1 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
1 -20 -20
5 -20 -20
0 -20 -20
0 -20 -20
1 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
2 -20 -20
0 -20 -20
6 -20 -20
2 -20 -20
0 -20 -20
2 -20 -20
4 -20 -20
4 -20 -20
1 -20 -20
0 -20 -20
0 -20 -20
7 -20 -20
0 -20 -20
7 -20 -20
7 -20 -20
7 -20 -20
7 -20 -20
5 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
1 -20 -20
6 -20 -20
7 -20 -20
7 -20 -20
7 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
7 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
0 -20 -20
1 -20 -20
0 -20 -20
2 -20 -20
6 -20 -20
7 -20 -20
7 -20 -20
7 -20 -20
7 -20 -20
7 -20 -20
7 -20 -20
7 -20 -20
0 -20 -20


KeyboardInterrupt: 

In [44]:
predictions

[[-0.8, -3.8, -1.7, 2.2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0],
 [-0.8, -3.8, -1.7, 2.2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1],
 [-0.8, -3.8, -1.7, 2.2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 2],
 [-0.8, -3.8, -1.7, 2.2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 3],
 [-0.8, -3.8, -1.7, 2.2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 4],
 [-0.8, -3.8, -1.7, 2.2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 5],
 [-0.8, -3.8, -1.7, 2.2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 6],
 [-0.8, -3.8, -1.7, 2.2, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 7]]

In [45]:
Qas.predict(predictions)

array([[ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan],
       [ nan]], dtype=float32)

In [52]:
Qas.predict(predictions)

array([[ 1.64997256],
       [ 1.56507087],
       [ 1.48738086],
       [ 1.43926144],
       [ 1.40296233],
       [ 1.36036527],
       [ 1.36534405],
       [ 1.40326011]], dtype=float32)

In [14]:
env.switch

True

In [12]:
env.p2_dead

AttributeError: 'Environment' object has no attribute 'p2_dead'

In [13]:
env.p2_score

0

In [14]:
env.p2_score_prev

1

In [14]:
env.__grab_features__()

p2_dead
player 2 dead?
apparently
100.0 -4.0


[100.0,
 -1.768,
 -6.64,
 -4.0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0.0,
 0,
 0.0,
 0]